<a href="https://colab.research.google.com/github/franciscojferrari/Amazing-Landscapes/blob/experiments/experiments_amazing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import tensorflow as tf
import importlib
import yaml
import os
from google.colab import auth
from pathlib import Path
from tqdm import tqdm 
from PIL import Image

In [2]:
!curl ipinfo.io

{
  "ip": "34.86.95.64",
  "hostname": "64.95.86.34.bc.googleusercontent.com",
  "city": "Washington",
  "region": "Washington, D.C.",
  "country": "US",
  "loc": "38.8951,-77.0364",
  "org": "AS15169 Google LLC",
  "postal": "20045",
  "timezone": "America/New_York",
  "readme": "https://ipinfo.io/missingauth"
}

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  90607      0 --:--:-- --:--:-- --:--:-- 90607
OK
74 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 74 not upgraded.
Need to get 10.8 MB of archives.
After this operation, 23.1 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.35.0_amd64.deb ...
Unpacking gcsfuse (0.35.0) ...
Setting up gcsfuse (0.35.0) ...


In [5]:
!mkdir dataset
!gcsfuse --implicit-dirs spade_dataset dataset

2021/05/22 13:14:31.905189 Using mount point: /content/dataset
2021/05/22 13:14:31.913349 Opening GCS connection...
2021/05/22 13:14:32.092950 Mounting file system "spade_dataset"...
2021/05/22 13:14:32.124940 File system has been successfully mounted.


In [6]:
GENERATED_VALIDATION_PATH = "/content/dataset/generated_images/cityscape/val/"

Create txt file with the path of the generated images

In [ ]:
import os
def get_url_list(files):
  for x in files:
    if x[-4:] == "jpeg":
      yield GENERATED_VALIDATION_PATH + x + '\n'

file_url = list(get_url_list(os.listdir(GENERATED_VALIDATION_PATH)))

In [ ]:
f = open(GENERATED_VALIDATION_PATH + "test_images.txt","w")
f.writelines(file_url)
f.close()

Using pre-trained model DRN to create segmentation mask for the generated images

In [ ]:
! git clone https://github.com/fyu/drn.git

Cloning into 'drn'...
remote: Enumerating objects: 203, done.
remote: Total 203 (delta 0), reused 0 (delta 0), pack-reused 203
Receiving objects: 100% (203/203), 521.11 KiB | 26.05 MiB/s, done.
Resolving deltas: 100% (103/103), done.


In [ ]:
import sys
! {sys.executable} -m pip install -r drn/requirements.txt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! chmod +x /content/dataset/generated_images/cityscape

In [ ]:
! python3 drn/segment.py test -d drn/datasets/cityscapes -l $GENERATED_VALIDATION_PATH \
-c 19 --arch drn_d_22 --pretrained /content/gdrive/MyDrive/cityscapes/drn_d_22_cityscapes.pth --phase test --batch-size 1

In [ ]:
for file in tqdm(os.listdir(PATH)):
  if file.endswith(".png"):
    dst_path = PATH + '.'.join(file.split('..'))
    os.rename(PATH + file, dst_path)

In [ ]:
PATH_gtmask = "/content/dataset/cityscape/gtfine/gtFine/val/"

In [ ]:
gt_masks = []
for (dir_path, _, file_names) in os.walk(PATH_gtmask):
  gt_masks.extend([dir_path+ "/" + file_mask for file_mask in file_names if file_mask.endswith("_gtFine_color.png")])
len(gt_masks)

500

In [ ]:
generated_masks = [PATH + mask for mask in os.listdir(PATH) if mask.endswith(".png")]
len(generated_masks)

500

In [ ]:
file_mask = list(zip(sorted(gt_masks), sorted(generated_masks)))
file_mask[0]

('/content/dataset/cityscape/gtfine/gtFine/val/frankfurt/frankfurt_000000_000294_gtFine_color.png',
 '/content/dataset/generated_images/cityscape/val/frankfurt_000000_000294.png')

In [ ]:
from PIL import Image
import numpy as np
import miou as m
import importlib
import pixelacc as p
from tqdm import tqdm_notebook as tqdm

importlib.reload(m)
importlib.reload(p)


miou = []
acc = []
fid = []
gen_list = []
gt_list = []
for gt_path, gen_path in tqdm(file_mask):
  gen_image = Image.open(gen_path)
  gen_array = np.asarray(gen_image)
  gt_image = Image.open(gt_path)
  gt_image = gt_image.convert("RGB").resize(gen_image.size,resample=Image.NEAREST)
  gt_array = np.asarray(gt_image)
  miou.append(m.calculate_miou(gen_array,gt_array))
  acc.append(p.pixel_accuracy(gen_array,gt_array))
  gen_list.append(gen_array)
  gt_list.append(gt_array)

gen_list = np.array(gen_list)
gt_list = np.array(gt_list)

Accuracy 0.12376666666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [ ]:
!pip install tensorflow_gan

     |████████████████████████████████| 368kB 31.9MB/s 


In [ ]:
#first_elem = [x[0] for x in file_mask]
#second_elem = [x[1] for x in file_mask]
import importlib
import FID as f

importlib.reload(f)

fid = f.run_fid(gen_list,gt_list)
print("FID is", fid)
m.calculate_miou(gen_list,gt_list)

#Final miou
miou = np.mean(miou)
acc = np.mean(acc)
print("MIOU: ",miou)
print("Accuracy", acc)

FID is 1367.5575210870832


0.13960056

## Experiment 2: Generate ground truth mask

In [ ]:
ORIGINAL_VALIDATION_PATH = "/content/dataset/cityscape/trainvaltest/leftImg8bit/val/"
file_url = []
for (dir_path, _, file_names) in os.walk(ORIGINAL_VALIDATION_PATH):
  file_url.extend([dir_path+ "/" + file_mask for file_mask in file_names if file_mask.endswith(".png")])
len(file_url)

500

In [ ]:
REDUCED_IMAGE_PATH = "/content/dataset/cityscape/reducedvalimages/"

In [ ]:

from PIL import Image
from tqdm import tqdm_notebook as tqdm

for image_path in tqdm(file_url):
  image = Image.open(image_path)
  reduced_image = image.resize((256, 128), resample=Image.NEAREST)
  reduced_image.save(REDUCED_IMAGE_PATH + image_path.split('/')[-1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
REDUCED_IMAGE_PATH = "/content/dataset/cityscape/reducedvalimages/"

file_url = []
for (dir_path, _, file_names) in os.walk(REDUCED_IMAGE_PATH):
  file_url.extend([dir_path+ "/" + file_mask + "\n" for file_mask in file_names if file_mask.endswith(".png")])
len(file_url)

500

In [ ]:
f = open(REDUCED_IMAGE_PATH + "test_images.txt","w")
f.writelines(file_url)
f.close()

In [ ]:
! python3 drn/segment.py test -d drn/datasets/cityscapes -l $REDUCED_IMAGE_PATH \
-c 19 --arch drn_d_22 --pretrained /content/gdrive/MyDrive/cityscapes/drn_d_22_cityscapes.pth --phase test --batch-size 1

In [ ]:
REDUCED_IMAGE_PATH = "/content/dataset/cityscape/reducedvalimages/"
GENERATED_VALIDATION_PATH = "/content/dataset/generated_images/cityscape/val/"

In [ ]:
gt_masks = [REDUCED_IMAGE_PATH + name for name in os.listdir(REDUCED_IMAGE_PATH) if name.endswith("_mask.png")]

generated_masks = [GENERATED_VALIDATION_PATH + name for name in os.listdir(GENERATED_VALIDATION_PATH) if name.endswith(".png")]

file_mask = list(zip(sorted(gt_masks), sorted(generated_masks)))
file_mask[0]

NameError: ignored

In [ ]:
original_images = [REDUCED_IMAGE_PATH + name for name in os.listdir(REDUCED_IMAGE_PATH) if not name.endswith("_mask.png")]

generated_images = [GENERATED_VALIDATION_PATH + name for name in os.listdir(GENERATED_VALIDATION_PATH) if not name.endswith(".png")]

file_images = list(zip(sorted(original_images), sorted(generated_images)))
file_images[0]

NameError: ignored

In [ ]:
from PIL import Image
import numpy as np
import miou as m
import importlib
import pixelacc as p
import FID as f
from tqdm import tqdm_notebook as tqdm

importlib.reload(m)
importlib.reload(p)
importlib.reload(f)

miou = []
acc = []
fid = []
gen_list = []
gt_list = []

for gt_path, gen_path in tqdm(file_mask):
  gen_image = Image.open(gen_path)
  gen_array = np.asarray(gen_image)
  gt_image = Image.open(gt_path)
  gt_image = gt_image.convert("RGB").resize(gen_image.size, resample=Image.NEAREST)
  gt_array = np.asarray(gt_image)
  miou.append(m.calculate_miou(gen_array, gt_array))
  acc.append(p.pixel_accuracy(gen_array, gt_array))
  gen_list.append(gen_array)
  gt_list.append(gt_array)

gen_list = np.array(gen_list)
gt_list = np.array(gt_list)

m.calculate_miou(gen_list,gt_list)
miou = np.mean(miou)
acc = np.mean(acc)
fid = f.run_fid(gen_list,gt_list)

print("MIOU: ",miou)
print("Accuracy", acc)
print("FID is", fid)

Accuracy 0.12573333333333334


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



87916544/87910968 [==============================] - 1s 0us/step
MIOU:  0.29728472
Accuracy 0.7544669189453125
FID is 336.93729124216304


## Experiment 3: FID between generated images and original images Cityscapes 

In [ ]:
#GENERATED_VALIDATION_PATH
#REDUCED_IMAGE_PATH

original_images = [REDUCED_IMAGE_PATH + name for name in os.listdir(REDUCED_IMAGE_PATH) if not name.endswith("_mask.png")]

generated_images = [GENERATED_VALIDATION_PATH + name for name in os.listdir(GENERATED_VALIDATION_PATH) if name.endswith(".jpeg")]

file_images = list(zip(sorted(original_images), sorted(generated_images)))

('/content/dataset/cityscape/reducedvalimages/frankfurt_000000_000294_leftImg8bit.png',
 '/content/dataset/generated_images/cityscape/val/frankfurt_000000_000294.jpeg')

In [ ]:
from PIL import Image
import numpy as np
import importlib
import FID as f
from tqdm import tqdm_notebook as tqdm

importlib.reload(f)

gen_list = []
gt_list = []

for gt_path, gen_path in tqdm(file_images):
  gen_image = Image.open(gen_path)
  gen_array = np.asarray(gen_image)
  gt_image = Image.open(gt_path)
  gt_image = gt_image.convert("RGB")
  gt_array = np.asarray(gt_image)
  gen_list.append(gen_array)
  gt_list.append(gt_array)

gen_list = np.array(gen_list)
gt_list = np.array(gt_list)
fid = f.run_fid(gen_list,gt_list)
print("FID is", fid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



87916544/87910968 [==============================] - 1s 0us/step
FID is 846.6181234309547


## Experiment 4: FID calculation for Celebs

In [6]:
# first we reduce the original images
PATH_generated = "/content/dataset/generated_images/faces/val/"
PATH_original = "/content/dataset/CelebAMask/CelebA-HG-img-Files/"

In [7]:
generated_images = [PATH_generated + image for image in os.listdir(PATH_generated)]

In [11]:
original_images = [PATH_original + image.split(".")[0] + ".jpg" for image in os.listdir(PATH_generated)]

In [14]:
file_images = list(zip(sorted(original_images), sorted(generated_images)))

In [23]:
from PIL import Image
import numpy as np
import importlib
import FID as f
from tqdm import tqdm_notebook as tqdm

importlib.reload(f)

gen_list = []
gt_list = []

for gt_path, gen_path in tqdm(file_images):
  gen_image = Image.open(gen_path)
  gen_array = np.asarray(gen_image)
  gt_image = Image.open(gt_path)
  gt_image = gt_image.convert("RGB").resize(gen_image.size)
  gt_array = np.asarray(gt_image)
  gen_list.append(gen_array)
  gt_list.append(gt_array)

gen_list = np.array(gen_list)
gt_list = np.array(gt_list)
fid = f.run_fid(gen_list,gt_list)
print("FID is", fid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



FID is 117.68668494747098


In [26]:
Image.open(file_images[0][0]).size

(1024, 1024)

In [27]:
Image.open(file_images[0][1]).size

(256, 256)

In [22]:
file_images[0]

('/content/dataset/CelebAMask/CelebA-HG-img-Files/10000.jpg',
 '/content/dataset/generated_images/faces/val/10000.jpeg')